In [2]:
import pandas as pd
from src.input.caiso_connector import (
    download_csv_file,
    unzip_csv,
    delete_data_files,
    find_csv_files,
    ZIP_DIRECTORY,
    target_url
)

# Download file

In [5]:
download_csv_file(target_url, ZIP_DIRECTORY)
unzip_csv(ZIP_DIRECTORY)

# Load dataframe 

In [18]:
for file in find_csv_files(ZIP_DIRECTORY):
    df = (pd.read_csv(file)
          .drop(columns=['OPR_DT', 'NODE_ID_XML']))

In [17]:
df.head()

,INTERVALSTARTTIME_GMT,INTERVALENDTIME_GMT,OPR_HR,NODE_ID,NODE,MARKET_RUN_ID,LMP_TYPE,XML_DATA_ITEM,PNODE_RESMRID,GRP_TYPE,POS,MW,OPR_INTERVAL,GROUP
0,2019-02-01T00:00:00-00:00,2019-02-01T00:05:00-00:00,17,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,RTM,MCC,LMP_CONG_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,-0.71137,1,1
1,2019-02-01T00:05:00-00:00,2019-02-01T00:10:00-00:00,17,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,RTM,MCC,LMP_CONG_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,-1.11032,2,1
2,2019-02-01T00:10:00-00:00,2019-02-01T00:15:00-00:00,17,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,RTM,MCC,LMP_CONG_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,-4.52263,3,1
3,2019-02-01T00:15:00-00:00,2019-02-01T00:20:00-00:00,17,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,RTM,MCC,LMP_CONG_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,-1.54268,4,1
4,2019-02-01T00:20:00-00:00,2019-02-01T00:25:00-00:00,17,AFPR_1_TOT_GEN-APND,AFPR_1_TOT_GEN-APND,RTM,MCC,LMP_CONG_PRC,AFPR_1_TOT_GEN-APND,ALL_APNODES,0,-2.21861,5,1


In [11]:
df['LMP_TYPE'].unique()

array(['MCC', 'MCE', 'MCL', 'LMP'], dtype=object)

In [12]:
df['MARKET_RUN_ID'].unique()

array(['RTM'], dtype=object)

In [13]:
df['OPR_HR'].unique()

array([17])

In [19]:
lmp_columns = ['INTERVALSTARTTIME_GMT', 'NODE','LMP_TYPE','MW']
for file in find_csv_files(ZIP_DIRECTORY):
    df = pd.read_csv(file, usecols=lmp_columns)
df.head()

,INTERVALSTARTTIME_GMT,NODE,LMP_TYPE,MW
0,2019-02-01T00:00:00-00:00,AFPR_1_TOT_GEN-APND,MCC,-0.71137
1,2019-02-01T00:05:00-00:00,AFPR_1_TOT_GEN-APND,MCC,-1.11032
2,2019-02-01T00:10:00-00:00,AFPR_1_TOT_GEN-APND,MCC,-4.52263
3,2019-02-01T00:15:00-00:00,AFPR_1_TOT_GEN-APND,MCC,-1.54268
4,2019-02-01T00:20:00-00:00,AFPR_1_TOT_GEN-APND,MCC,-2.21861
